# Trabalho 1 - Problema da Ponte e da Tocha

### Resolução de problema usando DFS

In [1]:
import networkx as nx
import time

In [2]:
Graph = nx.read_graphml("grafo_ponte_tocha.graphml")

In [3]:
def dfs(grafo, estado_inicial, estado_objetivo, limit=10000):
    pilha = [(estado_inicial, [estado_inicial])]
    visitados = set()

    while pilha:
        estado_atual, caminho = pilha.pop()

        if estado_atual in visitados:
            continue
        visitados.add(estado_atual)

        if estado_atual == estado_objetivo:
            return caminho, visitados

        if len(visitados) > limit:
            print("Limite de nós atingido.")
            break

        for vizinho in grafo.neighbors(estado_atual):
            if vizinho not in visitados:
                pilha.append((vizinho, caminho + [vizinho]))

    return None, visitados

In [4]:
estado_inicial = "inicio=ABCD|tocha=inicio"
estado_objetivo = "inicio=VAZIO|tocha=final"

In [5]:
start_time = time.time()
caminho, visitados = dfs(Graph, estado_inicial, estado_objetivo)
end_time = time.time()
duration = end_time - start_time

In [6]:
print("Caminho encontrado pela DFS:")
print(caminho)
print("\nNúmero de nós visitados:", len(visitados))
print(f"Tempo de execução: {duration}")

Caminho encontrado pela DFS:
['inicio=ABCD|tocha=inicio', 'inicio=CD|tocha=final', 'inicio=BCD|tocha=inicio', 'inicio=C|tocha=final', 'inicio=BC|tocha=inicio', 'inicio=VAZIO|tocha=final']

Número de nós visitados: 6
Tempo de execução: 0.00022268295288085938
